In [2]:
import mmcv
import cv2
import os
import glob
import sys
import numpy as np
import slidingwindow as sw
import pycocotools.mask as maskUtils

from mmdet.apis import init_detector, inference_detector
from skimage.measure import label, regionprops_table

# import shm_tools here
sys.path.append('..')
from shm_tools.shm_utils import imread, imwrite

In [ ]:
def connect_cracks(mask_output, epsilon = 200):
    '''
    :param mask_output: a numpy uint8 variable
    :param epsilon: distance between cracks to be connected
    :return: connect_mask_output : crack-connection result
    '''

    '''
    To-dos : 
    1 . Add iteration option
    2 . Add connection option considering a direction of a crack
    with the direction of ellipse of each crack 
    '''

    # label each crack
    labels, num = label(mask_output, connectivity=2, return_num=True)
    # get information of each crack area
    crack_region_table = regionprops_table(labels, properties=('label', 'bbox', 'coords', 'orientation'))

    width = crack_region_table['bbox-3'] - crack_region_table['bbox-1']
    height = crack_region_table['bbox-2'] - crack_region_table['bbox-0']

    crack_region_table['is_horizontal'] = width > height

    e2_list = []
    e1_list = []

    for crack_num, crack_region in enumerate(crack_region_table['label']):

        min_row = crack_region_table['bbox-0'][crack_num]
        min_col = crack_region_table['bbox-1'][crack_num]
        max_row = crack_region_table['bbox-2'][crack_num]-1
        max_col = crack_region_table['bbox-3'][crack_num]-1

        if crack_region_table['is_horizontal'][crack_num]:
            # max col / min col
            col = crack_region_table['coords'][crack_num][:, 1]

            e2 = crack_region_table['coords'][crack_num][np.argwhere(col == max_col), :][-1][0]
            e1 = crack_region_table['coords'][crack_num][np.argwhere(col == min_col), :][0][0]

            e2_list.append(e2)
            e1_list.append(e1)

        else:
            # max row / min row
            row = crack_region_table['coords'][crack_num][:, 0]

            e2 = crack_region_table['coords'][crack_num][np.argwhere(row == max_row), :][-1][0]
            e1 = crack_region_table['coords'][crack_num][np.argwhere(row == min_row), :][0][0]

            e2_list.append(e2)
            e1_list.append(e1)

    crack_region_table['e2'] = e2_list
    crack_region_table['e1'] = e1_list

    connect_line_img = np.zeros_like(mask_output, dtype=np.uint8)
    n = len(crack_region_table['label'])
    color = (1) # binary image

    for i in range(n):  # scan through all of crack area
        k_list = []
        for k in range(n):
            if not k == i: # compare with all the other cracks
                distance = []
                e_list = []

                # compare every direction
                # close-close / far-close / close-far / far-far
                for e1 in ['e1', 'e2']:
                    for e2 in ['e1', 'e2']:
                        d = np.subtract(crack_region_table[e1][k], crack_region_table[e2][i])
                        distance.append(np.sqrt(d[0] ** 2 + d[1] ** 2))
                        e_list.append([e1, e2])

                if not k_list: # when k_list is empty,
                    k_list.append([distance, e_list, k])

                elif np.min(k_list[0][0]) > np.min(distance):
                    k_list = []
                    k_list.append([distance, e_list, k])

        if k_list:

            if np.min(k_list[0][0]) < epsilon:
                dist_idx = np.argmin(k_list[0][0])
                e1, e2 = k_list[0][1][dist_idx]
                k_ = k_list[0][2]

                start_point = crack_region_table[e1][k_][::-1]
                end_point = crack_region_table[e2][i][::-1]

                connect_line_img = cv2.line(connect_line_img, tuple(start_point), tuple(end_point), color, 8)

    mask_output = mask_output + connect_line_img
    mask_output[mask_output > 1] = 1

    return mask_output